In [72]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
from sklearn.model_selection import KFold

In [73]:
df = pd.read_csv('/content/drive/MyDrive/NNFL-Assignment1/data_q4_q5.csv')
df.head()

,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,radius_se,texture_se,perimeter_se,area_se,smoothness_se,compactness_se,concavity_se,concave points_se,symmetry_se,fractal_dimension_se,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,diagnosis
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,1.0950,0.9053,8.589,153.40,0.006399,0.04904,0.05373,0.01587,0.03003,0.006193,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,M
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,0.5435,0.7339,3.398,74.08,0.005225,0.01308,0.01860,0.01340,0.01389,0.003532,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,M
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,0.7456,0.7869,4.585,94.03,0.006150,0.04006,0.03832,0.02058,0.02250,0.004571,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,M
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,0.4956,1.1560,3.445,27.23,0.009110,0.07458,0.05661,0.01867,0.05963,0.009208,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,M
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,0.7572,0.7813,5.438,94.44,0.011490,0.02461,0.05688,0.01885,0.01756,0.005115,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,M


In [74]:

df = df.drop_duplicates(keep='first')
#df = df.sample(frac = 1).reset_index(drop=True)
m = df.shape[0]
n = df.shape[1] - 1
print(m,n)

569 30


In [75]:
datan = df.values
X = datan[:, 0:30]
y = datan[:, -1]

#X = (X - np.min(X, axis = 0))/(np.max(X, axis=0)-np.min(X, axis=0))
print(X.shape, y.shape)

(569, 30) (569,)


In [76]:
num_classes = len(np.unique(y))
print(num_classes)

2


In [77]:
# M <-> 1   B <-> 2
def LRT(X_inst, X, y):
  prior_1 = 0
  prior_2 = 0
  m = len(y)
  n = X.shape[1]
  for i in y:
    if (i=='M'):
      prior_1 = prior_1 + 1
    if (i=='B'):
      prior_2 = prior_2 + 1
  prior_1 = prior_1/m
  prior_2 = prior_2/m
  X1 = np.array([X[i] for (i, label) in enumerate(y) if label == 'M'])
  X2 = np.array([X[i] for (i, label) in enumerate(y) if label == 'B'])
  u1 = np.mean(X1, axis=0)
  u2 = np.mean(X2, axis=0)
  covariance_matrix1 = np.cov(X1.T.astype(float))
  covariance_matrix2 = np.cov(X2.T.astype(float))
  coeff1 = 1/(((2*math.pi)**(n/2))*np.linalg.det(covariance_matrix1)**0.5)
  coeff2 = 1/(((2*math.pi)**(n/2))*np.linalg.det(covariance_matrix2)**0.5)
  likelihood1 = coeff1*np.exp(-0.5*np.dot(np.dot((X_inst - u1),np.linalg.inv(covariance_matrix1)),(X_inst - u1).T))
  likelihood2 = coeff2*np.exp(-0.5*np.dot(np.dot((X_inst - u2),np.linalg.inv(covariance_matrix2)),(X_inst - u2).T))
  if (likelihood2/prior_1) > (likelihood1/prior_2):
      return 'B'
  else:
      return 'M'

def confusion_matrix(predictions, labels):
  TP = 0
  FN = 0
  FP = 0
  TN = 0
  m = len(labels)

  for i in range(m):
      if labels[i] == 'B' and predictions[i] == 'B':
              TP = TP + 1
      if labels[i] == 'B' and predictions[i] == 'M':
              FN = FN + 1
      if labels[i] == 'M' and predictions[i] == 'B':
              FP = FP + 1
      if predictions[i] == 'M' and labels[i] == 'M':
              TN = TN + 1    
  return TP, FN, FP, TN

In [78]:
kf = KFold(n_splits=5)

accuracies, specificities, sensitivities = [], [], []
for train_index, test_index in kf.split(X):
  X_train, X_test = X[train_index], X[test_index]
  y_train, y_test = y[train_index], y[test_index]
  predictions = []
  for i in range(len(X_test)):
    predictions.append(LRT(X_test[i], X_train, y_train))
  TP, FN, FP, TN = confusion_matrix(predictions,y_test)
  acc = (TP+TN)/(TP+TN+FP+FN)
  spec = (TN)/(TN+FP)
  sens = (TP)/(TP+FN)
  accuracies.append(acc)
  specificities.append(spec)
  sensitivities.append(sens)

final_accuracy = sum(accuracies)/len(accuracies)
final_sensitivity = sum(sensitivities)/len(sensitivities)
final_specificity = sum(specificities)/len(specificities)

In [79]:
print("Mean accuracy = ", final_accuracy)
print("Mean sensitivity = ", final_sensitivity)
print("Mean specificity = ", final_specificity)

Mean accuracy =  0.9595714951094549
Mean sensitivity =  0.9675185667530671
Mean specificity =  0.9533377488926604


In [80]:
print("Max accuracy = ", max(accuracies))
print("Max specificity = ", max(sensitivities))
print("Max sensitivity = ", max(specificities))

Max accuracy =  0.9736842105263158
Max specificity =  0.9864864864864865
Max sensitivity =  1.0
